# Twitter Sentiment Analysis

This notebook aims to predict whether a person does or does not intend to get the COVID-19 vaccine by looking at Tweets.

## Libraries

In [1]:
import tweepy
import webbrowser
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from tweepy import OAuthHandler
from tweepy import API



## Gather data from Twitter API

In [2]:
# Developer API keys
consumer_key = 'bWejmjpTvWNkCiwpixz4iYxGK'
consumer_secret = 'Yambe1a5bWbsPoyKQg8MeQzdVHlcjnXzIm6k7uGDPsKvxyDki7'
access_token = '1374027292867969025-BcTPabYKmlcKQHZCkoXbZ2BvXXcbNs'
access_token_secret = 'STtqBirnsLcjoVkkRbtIpWLb55hTtyw6dJus8kwaxaqha'
callback_uri = 'oob'

In [3]:
auth = OAuthHandler(consumer_key, consumer_secret, callback_uri)
auth.set_access_token(access_token, access_token_secret)

In [17]:
api = API(auth, wait_on_rate_limit= True, wait_on_rate_limit_notify= True)

In [ ]:
# show my Twitter handle
me = api.me()
print(me.screen_name)

In [18]:
# simple query to get the hang of it
query = '#vaccine covid'
for i, status in enumerate(tweepy.Cursor(api.search, q= query).items(50)):
    print (i, status.text)

# filter only English tweets    
# query = '#vaccine covid'
# for i, status in enumerate(tweepy.Cursor(api.search, q= query, lang= 'en').items(50)):
#     print (i, status.text)

0 RT @CaulfieldTim: "The impact of #COVID #vaccine misinformation is not trivial."

Poll: The misinformed are less likely to get vaccinated h…
1 Please open the link and book your appointment for the vaccine #Covid19 #Vaccine @Ottawa @Canada 
https://t.co/zYcGzCbT0D
2 RT @PoliticsPollss: Do you think children under 16 should or should not be vaccinated against Covid-19? #Vaccine #Coronavirus #Politics
3 RT @miamicheap: Where to get a COVID-19 #vaccine in #Miami. Details here: https://t.co/CNuiIYmihP https://t.co/LuZjCHrbk0
4 Health authorities across the Arab world are reporting an alarming rise in coronavirus infections as the fasting mo… https://t.co/auiWJCHwiU
5 RT @DrLeanaWen: Great news from Pfizer this morning! Their #covid19 #vaccine is efficacious for at least 6 months. Appears to work very wel…
6 COVID-19 vaccination clinic delivers AstraZeneca vaccine to Saint John
#Health #Astrazeneca #BloodClots #Clinic… https://t.co/Iwfr2RP2Nq
7 RT @tjcrawfordCVS: 10 million+ #COVID19 #vac

In [27]:
# collect 5000 tweets 
number_of_tweets = 5000
tweets = []
language = []
time = []

# only collect full text, language, and time it was posted
# remove any retweets '-RT'
for i in tweepy.Cursor(api.search, q= 'vaccine covid -RT',  
                       tweet_mode = 'extended').items(number_of_tweets):
    tweets.append(i.full_text)
    language.append(i.lang)
    time.append(i.created_at)

In [28]:
# turn tweets / lists into dataframe
df = pd.DataFrame({'time': time, 'language': language, 'tweets': tweets})

In [29]:
# save df to csv
df.to_csv('tweets.csv', index= False)

Collect more tweets after waiting to avoid collecting the same tweets.

In [50]:
# create function to get more tweets and add to existing csv
def get_more_tweets(num_tweets):
    number_of_tweets = num_tweets
    tweets = []
    language = []
    time = []

    # only collect full text, language, and time it was posted
    # remove any retweets '-RT'
    for i in tweepy.Cursor(api.search, q= 'vaccine covid -RT',  
                           tweet_mode = 'extended').items(number_of_tweets):
        tweets.append(i.full_text)
        language.append(i.lang)
        time.append(i.created_at)
        
    # turn tweets / lists into dataframe
    df = pd.DataFrame({'time': time, 'language': language, 'tweets': tweets})
    
    # add to existing csv file
    df.to_csv('tweets.csv', mode= 'a', index= False, header= False)

In [51]:
get_more_tweets(100)

### Search for Tweets from 30 Days Ago

In [46]:
# function to turn results into DataFrame
def extract_text_as_df(text_list):
    columns = set()
    allowed_types = [str, int]
    tweets_data = []
    for status in text_list:
        status_dict = dict(vars(status))
        keys = status_dict.keys()
        single_tweet_data = {"author": status.author.screen_name}
        for k in keys:
            try:
                v_type = type(status_dict[k])
            except:
                v_type = None
            if v_type != None:
                if v_type in allowed_types:
                    single_tweet_data[k] = status_dict[k]
                    columns.add(k)
        tweets_data.append(single_tweet_data)


    header_cols = list(columns)
    header_cols.append('author')
    df = pd.DataFrame(tweets_data, columns=header_cols)
    return df

In [41]:
query = 'vaccine covid -RT'
results_30 = api.search_30_day('datacollection', query, maxResults= 100)
# results_30
extract_text_as_df(results_30)

[Status(_api=<tweepy.api.API object at 0x7fc9dca19490>, _json={'created_at': 'Thu Apr 01 22:45:42 +0000 2021', 'id': 1377754069276758017, 'id_str': '1377754069276758017', 'text': 'New COVID-19 vaccine clinic opens up in Gonzales County, first doses to be given April 7th https://t.co/ECEXyytrkS https://t.co/0ROlzIpp4D', 'display_text_range': [0, 114], 'source': '<a href="http://twibble.io" rel="nofollow">Twibble.io</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 17194257, 'id_str': '17194257', 'name': 'News 4 San Antonio', 'screen_name': 'News4SA', 'location': 'San Antonio, Texas', 'url': 'http://news4sa.com', 'description': "We are News 4 San Antonio. Your source for the latest news, weather and traffic. We're also San Antonio’s first TV station and an NBC affiliate.", 'translator_type': 'none', 'protected': False, 'verified': True, '

In [44]:
old_results_df = extract_text_as_df(results_30)

In [45]:
old_results_df

quoted_status_id in_reply_to_status_id_str in_reply_to_screen_name lang  \
0                NaN                       NaN                     NaN   en   
1                NaN       1377751430325493772                DrTedros   en   
2                NaN                       NaN                     NaN   en   
3                NaN                       NaN                     NaN   en   
4                NaN                       NaN                     NaN   en   
..               ...                       ...                     ...  ...   
95               NaN                       NaN                     NaN   en   
96               NaN       1377300368426668034         sairish_hussain   en   
97               NaN                       NaN                     NaN   en   
98               NaN                       NaN                     NaN   en   
99               NaN                       NaN                     NaN   en   

                 source quoted_status_id_str  favorite_count  \
0            Twibble.io                  NaN               0   
1       Twitter Web App                  NaN               0   
2             TweetDeck                  NaN               2   
3             Instagram                  NaN               0   
4       Twitter Web App                  NaN               0   
..                  ...                  ...             ...   
95  Twitter for Android                  NaN               0   
96   Twitter for iPhone                  NaN               0   
97      Twitter Web App                  NaN               0   
98  Twitter for Android                  NaN               0   
99   Twitter for iPhone                  NaN               1   

                                                 text  reply_count  \
0   New COVID-19 vaccine clinic opens up in Gonzal...            0   
1   @DrTedros @presidentaz @WHO When is WHO planni...            0   
2   Free COVID-19 Vaccination events for Arizona e...            0   
3   Headed to my first concert post-Covid vaccine!...            0   
4   How mRNA works in covid vaccine.\n\nmessenger ...            0   
..                                                ...          ...   
95  2021-04-01\nI got a second Covid-19 vaccine to...            0   
96  @sairish_hussain @Mel_Pickup @BTHFT Thanks for...            0   
97  Does anyone know what covid-19 vaccine that Ch...            0   
98  Smallpox ‘virus squads’ and the mandatory vacc...            0   
99  Y’all scared to take the covid vaccine yet be ...            0   

    in_reply_to_status_id                   id filter_level  quote_count  \
0                     NaN  1377754069276758017          low            0   
1            1.377751e+18  1377754063786274816          low            0   
2                     NaN  1377754062167236608          low            0   
3                     NaN  1377754043926319112          low            0   
4                     NaN  1377754008140578818          low            0   
..                    ...                  ...          ...          ...   
95                    NaN  1377753368194584581          low            0   
96           1.377300e+18  1377753366995075079          low            0   
97                    NaN  1377753364277030913          low            0   
98                    NaN  1377753331817398272          low            0   
99                    NaN  1377753326633115648          low            0   

    in_reply_to_user_id  retweet_count  \
0                   NaN              0   
1           189868631.0              0   
2                   NaN              0   
3                   NaN              0   
4                   NaN              0   
..                  ...            ...   
95                  NaN              0   
96          611824539.0              0   
97                  NaN              0   
98                  NaN              0   
99                  NaN              0   

                           

In [34]:
number_of_tweets = 50
tweets = []
language = []
time = []
query = 'vaccine covid -RT'

for i in tweepy.Cursor(api.search_30_day, 'datacollection', query, 
                       maxResults= number_of_tweets):
    tweets.append(i.full_text)
    language.append(i.lang)
    time.append(i.created_at)

TypeError: 'Cursor' object is not iterable

## EDA

In [ ]:
df = pd.read_csv('tweets.csv')

In [ ]:
lang_df = df.sort_values('language')

In [ ]:
plt.figure(figsize= (12, 8))
lang_df.language.value_counts(normalize=True).plot(kind= 'barh')
plt.title('Proportion of Languages Collected')
plt.xlim(-.01, 1)
plt.show()

In [ ]:
lang_df.language.value_counts(normalize=True)

In [13]:
##translate all to english?

## Text Sentiment
---
Does the tweet have a positive, negative, or neutral tone?

In [16]:
# do text sentiment